# Log Analytics ワークスペースをノートブックと統合する

https://learn.microsoft.com/ja-jp/azure/azure-monitor/logs/notebooks-azure-monitor-logs

## Log Analytics ワークスペースの ID に設定

In [1]:
LOGS_WORKSPACE_ID = "DEMO_WORKSPACE"

## Azure Monitor ログの認証とクエリを実行するように `LogsQueryClient` を設定

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.core.pipeline.policies import AzureKeyCredentialPolicy
from azure.identity import DefaultAzureCredential
from azure.monitor.query import LogsQueryClient

if LOGS_WORKSPACE_ID == "DEMO_WORKSPACE":
    credential = AzureKeyCredential("DEMO_KEY")
    authentication_policy = AzureKeyCredentialPolicy(name="X-Api-Key", credential=credential)
else:
    credential = DefaultAzureCredential()
    authentication_policy = None

logs_query_client = LogsQueryClient(credential, authentication_policy=authentication_policy)

### Log Analytics ワークスペースで特定のクエリを実行し、結果を Pandas DataFrame として返す query_logs_workspace と呼ばれるヘルパー関数を定義

In [3]:
import pandas as pd
import plotly.express as px

from azure.monitor.query import LogsQueryStatus
from azure.core.exceptions import HttpResponseError


def query_logs_workspace(query):
    try:
        response = logs_query_client.query_workspace(LOGS_WORKSPACE_ID, query, timespan=None)
        if response.status == LogsQueryStatus.PARTIAL:
            error = response.partial_error
            data = response.partial_data
            print(error.message)
        elif response.status == LogsQueryStatus.SUCCESS:
            data = response.tables
        for table in data:
            my_data = pd.DataFrame(data=table.rows, columns=table.columns)        
    except HttpResponseError as err:
        print("something fatal happened")
        print (err)
    return my_data

### ノートブックで Log Analytics ワークスペースからデータを探索して視覚化

In [ ]:
TABLE = "Usage"

QUERY = f"""
let starttime = 7d; // Start date for the time series, counting back from the current date
let endtime = 0d; // today 
{TABLE} | project TimeGenerated, DataType, Quantity 
| where TimeGenerated between (ago(starttime)..ago(endtime))
| summarize ActualUsage=sum(Quantity) by TimeGenerated=bin(TimeGenerated, 1h), DataType
"""

df = query_logs_workspace(QUERY)
display(df)

### Pandas DataFrame に基づいて、時間の経過に伴うさまざまなデータ型の時間単位の使用状況を示すグラフとしてデータを表示

In [ ]:
df = df.sort_values(by="TimeGenerated")
graph = px.line(df, x='TimeGenerated', y="ActualUsage", color='DataType', title="Usage in the last week - All data types")
graph.show()